# Lab 2 - Frequent Itemset Mining

- Bài tập rèn luyện về khả năng lập trình thuật toán khai thách mẫu phổ biến (Frequent Itemset Mining)

- **Trong bài tập này, sinh viên không được phép dùng các thư viện hỗ trợ cài đặt thuật toán FIM như sklearn, mlxtend, ...**


## Thông tin cá nhân 
- Họ và tên: Nguyễn Tấn Phát
- Mã số sinh viên: 20127588 

&#127802; Sinh viên lưu ý mình sẽ dùng chương trình hỗ trợ chấm bài nên bạn cần phải tuân thủ chính xác qui định mà mình đặt ra, nếu không rõ thì hỏi, chứ không nên tự tiện làm theo ý của cá nhân.

## Cách làm bài

**Bạn sẽ làm trực tiếp trên file Python notebook này (jupyter notebook - sinh viên không được dùng Google Colab).**

Đầu tiên, bạn điền họ tên và MSSV vào phần đầu file ở bên trên. Trong file, bạn làm bài ở những chỗ có ghi là:

```python
# YOUR CODE HERE
raise NotImplementedError()
```
hoặc đối với những phần code không bắt buộc thì là:
```python
# YOUR CODE HERE (OPTION)
```

Tất nhiên, khi làm thì bạn xóa dòng `raise NotImplementedError()` đi.
Đối những phần yêu cầu code thì thường ở ngay phía dưới sẽ có một (hoặc một số) cell chứa các bộ test để giúp bạn biết đã code đúng hay chưa; nếu chạy cell này không có lỗi gì thì có nghĩa là qua được các bộ test. Trong một số trường hợp, các bộ test có thể sẽ không đầy đủ; nghĩa là, nếu không qua được test thì là code sai, nhưng nếu qua được test thì chưa chắc đã đúng hoàn toàn.

Trong khi làm bài, bạn có thể cho in ra màn hình, tạo thêm các cell để test. Nhưng khi nộp bài thì bạn xóa các cell mà bạn tự tạo, xóa hoặc comment các câu lệnh in ra màn hình. Bạn lưu ý <font color=red>không được tự tiện xóa các cell hay sửa code của Thầy</font> (trừ những chỗ được phép sửa như đã nói ở trên).


Trong khi làm bài, thường xuyên `Ctrl + S` để lưu lại bài làm của bạn, tránh mất mát thông tin.

Bạn có thể thảo luận ý tưởng với bạn khác hoặc từ các nguồn khác, nhưng code và bài làm phải là của bạn, dựa trên sự hiểu thật sự của bạn. Nếu vi phạm thì sẽ bị 0 điểm cho toàn bộ môn học.

## Cách nộp bài

Khi chấm bài, đầu tiên mình sẽ chọn `Kernel` - `Restart & Run All` để restart và chạy tất cả các cell trong notebook của bạn; do đó, trước khi nộp bài, bạn nên chạy thử `Kernel` - `Restart & Run All` để đảm bảo mọi chuyện diễn ra đúng như mong đợi.

Sau đó, bạn tạo thư mục nộp bài theo cấu trúc sau:
- Thư mục `MSSV` (vd, nếu bạn có MSSV là 123 thì bạn đặt tên thư mục là `123`)
    - File `FIM - SV.ipynb`
- Các bạn nén lại MSSV.zip (*.zip chứ không phải .rar hay gì khác)

Cuối cùng, bạn nén thư mục `MSSV` này lại và nộp ở link trên moodle. <font color=red>Bạn lưu ý tuân thủ chính xác qui định nộp bài.</font>

In [24]:
import os
import numpy as np
import csv
# YOUR CODE HERE (OPTION)

- Trong bài tập này, nhiệm vụ của chúng ta là khai thác các tập phổ biến (Frequent Itemset Mining).
- Bộ dữ liệu chúng ta dùng sẽ là dữ liệu mua hàng trong một siêu thị.

# 1. Đọc và khám phá dữ liệu

## a. Đọc dữ liệu

- Đầu tiên, ta sẽ đọc dữ liệu và lưu vào một `python list` 2D có tên là `list_transactions`. Dữ liệu nằm trong file `groceries.csv`
- Ví dụ"
`list_transactions` = [
                    ['milk', 'coffee'],
                    ['vegetable', 'candy', 'tomato']
                    ...
                 ]
                 
### Lưu ý:
- Trong bài tập này, ta sẽ xem transaction_id là thứ tự của transaction đó trong `list_transactions`.
- Đối với phần tử trong `list_transactions`, ta cần gọi hàm sorted() của Python. Hành động này để đảm bảo mọi phần tử được sắp xếp theo một thứ tự.
    + Ví dụ: `list_transactions[i] = sorted(list_transactions[i])`

In [25]:
list_transactions = []

# YOUR CODE HERE
f = open('groceries.csv', 'r')
line = f.readline()
while line:
    line = line.replace('\n', '')
    items = sorted(line.split(','))
    list_transactions.append(items)
    line = f.readline()
f.close()

In [26]:
print(list_transactions[0])

['citrus fruit', 'margarine', 'ready soups', 'semi-finished bread']


In [27]:
# TEST
assert len(list_transactions) == 9835
assert list_transactions[0] == ['citrus fruit', 'margarine', 'ready soups', 'semi-finished bread']
assert list_transactions[-1] == ['chicken', 'other vegetables', 'shopping bags', 'tropical fruit', 'vinegar']

## b. Khám phá dữ liệu

- Tiếp theo, ta sẽ tìm tất cả item trong dữ liệu. Kết quả sẽ được lưu vào `1D python list` có tên là `list_items`.
- Ví dụ: `list_items` = ['milk', 'coffee', ...]
- **Lưu ý**: để dữ liệu của mình và các bạn giống nhau, sau khi xử lý xong `list_items`, các bạn gọi hàm `sorted()` của Python. 
Ví dụ: `list_items = sorted(list_items)`

In [28]:
list_items = []

# YOUR CODE HERE
dic_items = {}
for transaction in list_transactions:
    for item in transaction:
        if item not in dic_items:
            dic_items[item] = 1
        else:
            dic_items[item] += 1

list_items = sorted(list(dic_items.keys()))


In [29]:
# TEST
assert list_items == sorted(list((set(list_items))))
assert len(list_items) == 169

# 2. Khái niệm cơ bản - khai thác mẫu phổ biển (Frequent itemset mining)

- Trong phần này, chúng ta sẽ tính một số khái niệm cơ bản trong khai thác mẫu phổ biến gồm:
    + Itemset_Cover(X): tập hợp các transaction chứa X.
    + Absolute_Support(X): số lượng các transaction chứa X.
    + Relative_Support(X): tỷ lệ transaction chứa X.
    + Frequent_itemset(X): itemset X là frequent (phổ biến) nếu Frequent_itemset(X) $\geq$ min_sup 

## a. Itemset Cover

- Itemset_Cover(X): tập hợp các transaction chứa X.

- Nhiệm vụ của sinh viên: hoàn thành hàm `Find_Itemset_Cover(X, list_transactions)` bên dưới

In [30]:
def Find_Itemset_Cover(X, list_transactions):
    """
    Function find the set of transaction containing X.
    
    * Parameter:
    X -- a 1D python list, indicate an itemset. For example ['milk', 'coffee', ...]
    list_transactions - a 2D python list, indicate the list of transactions
    
    * Return:
    list_transaction_cover -- a python list, indicate the list of transaction_id containing X. 
                                For example: [1, 2, 100, ...]
    """
    list_transaction_cover = []
    
    # YOUR CODE HERE
    for i, transaction in enumerate(list_transactions):
        if set(X).issubset(set(transaction)):
            list_transaction_cover.append(i)    
    return list_transaction_cover

In [31]:
# TEST

# ----- Test case 1 -----
X = ['sausage', 'whole milk', 'yogurt', 'coffee']
list_transaction_cover = Find_Itemset_Cover(X, list_transactions)
assert list_transaction_cover == [103, 1252, 2121, 3294, 3500, 4430, 4454, 6617, 6710, 8367, 8813]

# ----- Test case 2 -----
X = ['onions', 'domestic eggs', 'soda', 'hygiene articles']
list_transaction_cover = Find_Itemset_Cover(X, list_transactions)
assert list_transaction_cover == [267, 1000]

## b. Absolute support

- Absolute_Support(X): số lượng các transaction chứa X.
- Nhiệm vụ của sinh viên, hoàn thành hàm `Cal_Absolute_Support(X, list_transactions)` bên dưới.

In [32]:
def Cal_Absolute_Support(X, list_transactions):
    """
    Function calculate absolute support of itemset X.
    
    * Parameter:
    X -- a 1D python list, indicate an itemset. For example ['milk', 'coffee', ...]
    list_transactions - a 2D python list, indicate the list of transactions
    
    * Return:
    abs_support -- an integer, indicate the number of transactions containing X
    """
    
    abs_support = None
    
    # YOUR CODE HERE
    cover_list = Find_Itemset_Cover(X, list_transactions)
    abs_support = len(cover_list)
    
    return abs_support

In [33]:
X = ['yogurt']
abs_support = Cal_Absolute_Support(X, list_transactions)
print(abs_support)

1372


In [34]:
# TEST

# ----- Test case 1 -----
X = ['sausage', 'whole milk', 'yogurt', 'coffee']
abs_support = Cal_Absolute_Support(X, list_transactions)
assert abs_support == 11

# ----- Test case 2 -----
X = ['onions', 'domestic eggs', 'soda', 'hygiene articles']
abs_support = Cal_Absolute_Support(X, list_transactions)
assert abs_support == 2

## c. Relative_Support

- Relative_Support(X): tỷ lệ transaction chứa X.

- Equation:
$$ Relative = \frac{Absolute}{|transaction|}$$


- Nhiệm vụ của sinh viên: hoàn thành hàm `Cal_Relative_Support(X, list_transactions)` bên dưới.
- **Lưu ý**: kết quả làm tròn 4 chữ số thập phân, dùng hàm `round(x, 4)` của Python

In [35]:
def Cal_Relative_Support(X, list_transactions):
    """
    Function calculate absolute support of itemset X.
    
    * Parameter:
    X -- a 1D python list, indicate an itemset. For example ['milk', 'coffee', ...]
    list_transactions - a 2D python list, indicate the list of transactions
    
    * Return:
    relative_support -- a float, indicate the probability of transactions containing X
    """
    
    relative_support = None
    
    # YOUR CODE HERE
    absolute = Cal_Absolute_Support(X, list_transactions)
    relative_support = absolute / len(list_transactions)    
    return round(relative_support, 4)

In [36]:
# TEST

# ----- Test case 1 -----
X = ['sausage', 'whole milk', 'yogurt', 'coffee']
relative_support = Cal_Relative_Support(X, list_transactions)
assert relative_support == 0.0011

# ----- Test case 2 -----
X = ['onions', 'domestic eggs', 'soda', 'hygiene articles']
relative_support = Cal_Relative_Support(X, list_transactions)
assert relative_support == 0.0002

# ----- Test case 3 -----
X = ['bottled water']
relative_support = Cal_Relative_Support(X, list_transactions)
assert relative_support == 0.1105

## d. Frequent_itemset(X): 
- Gọi itemset X là frequent (phổ biến) nếu Frequent_itemset(X) $\geq$ min_sup 
- Nhiệm vụ của sinh viên, hoàn thành hàm `Check_Frequent_Itemset(X, list_transactions)` bên dưới.

In [37]:
def Check_Frequent_Itemset(X, list_transactions, min_sup):
    """
    Function check the itemset X is a frequent of not
    
    * Parameter:
    X -- a 1D python list, indicate an itemset. For example ['milk', 'coffee', ...]
    list_transactions -- a 2D python list, indicate the list of transactions
    min_support -- a float, indicate the minimum support value
    
    * Return:
    is_frequent -- a boolean value. True if X is frequent. False otherwise.
    """
    
    is_frequent = None
    
    # YOUR CODE HERE
    frequent_itemset = Cal_Relative_Support(X, list_transactions)
    if frequent_itemset >= min_sup:
        is_frequent = True
    else:
        is_frequent = False
    
    return is_frequent

In [38]:
# TEST

# ----- Test case 1 -----
X = ['whole milk']
min_sup = 0.15
is_frequent = Check_Frequent_Itemset(X, list_transactions, min_sup)
assert is_frequent == True

# ----- Test case 1 -----
X = ['soda']
min_sup = 0.15
is_frequent = Check_Frequent_Itemset(X, list_transactions, min_sup)
assert is_frequent == True

## e. Một số khái niệm khác

- itemset:
    + Tập các item được lấy ra từ list_items.
    + Ví dụ: ['Butter', 'Bread', 'Milk'] hoặc ['Butter', 'Bread']

- k-itemset: 
    + itemset có kích thước k. 
    + Ví dụ: ['milk', 'coffee', 'canday'] là 3-itemset
    

# 3. Bài toán Khai thác mẫu phổ biển (Frequent itemset mining)

- Xét bài toán gồn:
    + list_items (ký hiệu I) -- tập hợp tất cả các item trong siêu thị.
    + list_transactions -- tập hợp tất cả các transaction trong siêu thị.
    + min support s -- giá trị ngưỡng support nhỏ nhất.
    
- Mục tiêu: tìm tất cả các itemset X có thể có, sao cho relative_support(X) $\geq$ min_support

## a. Cách tiếp cận để giải bài toán FIM

- Cách đơn giản nhất để giải bài toán FIM:
    + Gọi I là kích thước của `list_items`.
    + Duyệt k: 1, 2, ..., I:
        + Tìm tất cả k-itemset có relative support lớn hơn min_support.
        
- Số lần duyệt của bài toán trên là: $2^I - 1$. Một con số vô cùng lớn và không khả thi trong quá trình tính toán.

- Do đó, ta sẽ dùng thuật toán Apriori để giải bài toán FIM.

## b. Apriori algorithm

- Các bước chính của thuật toán Apriori gồm:
    1. Tạo các tập phổ biến 1-itemset ($L_1$)
    2. Tạo ra tập ứng viên (gọi là $C_{k+1}$), gồm các (k+1)-itemset từ tập $L_k$.
    3. Kiểm tra các ứng viên trong $C_{k+1}$ có là tập phổ biến không. Nếu là phổ biến thì lưu vào $L_{k+1}$.
    4. Lặp lại bước 2 cho đến khi không còn ứng viên nào.
    
    
- Trong quá trình trên, bước 1 và bước 4 là các bước đơn giản. Tuy nhiên, bước 2 và bước 3 là các bước phức tạp và mang tính chất quan trọng của thuật toán Apriori. Do đó, ta sẽ phân tích kỹ bước 2 và bước 3.

### **Bước 2 (bước tổng hợp - join step):**
- Bước này có nhiệm vụ tạo ra tập ứng viên $C_{k+1}$.
- $C_{k+1}$ được tạo từ tập $L_k$ bằng phép tính self-join: $C_{k+1} = L_k * L_k$
- Hai k-itemset có thể self-join nếu chúng có cùng tiền tố. (k-1 item đầu tiên).
    + Ví dụ: ['candy', 'coffee', 'milk'] và ['candy', 'coffee', 'fruit'] sẽ tạo thành [coffee, milk, candy, fruit]
    
### Ví dụ: 

- Bước tổng hợp
    + $L_3$ = [abc , abd , acd , ace, bcd]
    + Bước tổng hợp: $C_4$ = [abcd, acde]
    + Vì: abcd = abc * abd, acde = acd * ace
   

- Nhiệm vụ của sinh viên: hoàn thành hàm `Cal_Self_Join(itemset_1, itemset_2)`. 
- **Lưu ý:** kết quả cần được gọi hàm sorted(), để đảm bảo thứ tự của itemset. 

In [39]:
def Cal_Self_Join(itemset_1, itemset_2):
    """
    Function perform self-joining step between itemset_1 and itemset_2
    
    * Parameter:
    itemset_1 -- a 1D python list, indicate the itemset. For example ['candy', 'coffee', 'fruit']
    itemset_2 -- a 1D python list, indicate the itemset. For example ['candy', 'coffee', 'milk']
    
    * Return:
    join_itemset -- a 1D python list, indicate the join itemset. For example ['candy', 'coffee', 'fruit', 'milk'].
                        In case itemset_1 and itemset_2 can not join. Return []
    """
    
    join_itemset = None
    
    # YOUR CODE HERE
    itemset_3 = itemset_1 + itemset_2
    itemset_3 = list(set(itemset_3))
    if len(itemset_3) == len(itemset_1) + 1:
        join_itemset = itemset_3
    else:
        join_itemset = []

    
    return sorted(join_itemset)

In [40]:
# TEST

# ----- Test case 1 -----
itemset_1 = ['candy', 'coffee', 'milk']
itemset_2 = ['candy', 'coffee', 'fruit']
assert Cal_Self_Join(itemset_1, itemset_2) == ['candy', 'coffee', 'fruit', 'milk']

# ----- Test case 2 -----
itemset_1 = ['candy', 'coffee', 'milk']
itemset_2 = ['candy', 'coffee']
assert Cal_Self_Join(itemset_1, itemset_2) == []

### **Bước 3 (bước cắt tỉa -- prune step):**

- Bước này có nhiệm vụ: loại bỏ các ứng viên $C_{k+1}$ và trả về $L_{k+1}$.
- Ý tưởng đơn giản: loại bỏ các ứng viên có relative support nhỏ hơn min_sup. Tuy nhiên, các này yêu cầu truy xuất dữ liệu list transaction => Không hiệu quả.

- Ta thực hiện cắt tỉa dựa vào hai yếu tố sau:
1. Cắt tỉa bởi tính chất của Apriori:
    + Nếu ứng viên k-itemset có tập con (k-1) không phải là tập phổ biến. Thì ứng viên k-itemset đó không phải là phổ biến => Loại.
    + Tại bước này, ta chỉ cần kiểm tra (k-1) tập con.
    
    
2. Cắt tỉa bởi min_sup:
    + Ứng viên nào có relative support nhỏ hơn min_sup thì loại.
    
### Ví dụ: 

- Bước tổng hợp
    + $L_3$ = [abc , abd , acd , ace, bcd]
    + Bước tổng hợp: $C_4$ = [abcd, acde]
    + Vì: abcd = abc * abd, acde = acd * ace
    
- Bước cắt tỉa:
    + Cắt tỉa bởi Apriori: acde bị cắt tỉa, do cde không thuộc $L_3$
    + Cắt tỉa bởi min_sup: kiểm tra relative support của abcd có lớn hơn min_sup?

- Nhiệm vụ của sinh viên: hoàn thành hàm `Prune_By_Apriori(list_candidate, list_previous_frequent_itemset)` được định nghĩa bên dưới.
- **Lưu ý:** mỗi ứng viên (candidate) trong `new_list_candidate`, đều cần được gọi hàm `sorted()` của Python. Và `new_list_candidate` cũng cần được gọi hàm `sorted()` của Python

In [41]:
def Prune_By_Apriori(list_candidate, list_previous_frequent_itemset):
    """
    Function prune by Apriori 
    
    * Parameter:
    list_candidate -- a 2D python list, indicate the C_(k+1)
    list_previous_frequent_itemset -- a 2D python list, indicate the L_k
    
    * Return:
    new_list_candidate -- a 2D python list, indicate the C_(k+1) after prune redundant candidate.
    """
    
    new_list_candidate = []
    
    # YOUR CODE HERE

    new_list_candidate = []
    for candidate in list_candidate:
        lst = []
        for itemset in list_previous_frequent_itemset:
            if set(itemset).issubset(set(candidate)):
                lst.append(itemset)
        if len(lst) == len(candidate):
            new_list_candidate.append(candidate)
            
    return new_list_candidate

In [42]:
# TEST CASE 1

list_candidate = [
    ['a', 'b', 'c', 'd'],
    ['a', 'c', 'd', 'e']
]

list_previous_frequent_itemset = [
    ['a', 'b', 'c'],
    ['a', 'b', 'd'],
    ['a', 'c', 'd'],
    ['a', 'c', 'e'],
    ['b', 'c', 'd']
]

test_1 = Prune_By_Apriori(list_candidate, list_previous_frequent_itemset)
print(test_1)
assert test_1 == [['a', 'b', 'c', 'd']]

[['a', 'b', 'c', 'd']]


In [43]:
# TEST CASE 2

list_candidate = [
    ['a', 'b', 'c'],\
    ['a', 'b', 'd'],\
    ['a', 'b', 'e'],\
    ['a', 'c', 'd'],\
    ['a', 'c', 'e'],\
    ['a', 'd', 'e'],\
    ['b', 'c', 'e']
]

list_previous_frequent_itemset = [
    ['a', 'b'],\
    ['a', 'c'],\
    ['a', 'd'],\
    ['a', 'e'],\
    ['b', 'c'],\
    ['b', 'e'],\
    ['c', 'd'],\
    ['c', 'e'],\
    ['d', 'e'],\
    ['e', 'f']
]

test_2 = Prune_By_Apriori(list_candidate, list_previous_frequent_itemset)
assert test_2 == [
    ['a', 'b', 'c'],\
    ['a', 'b', 'e'],\
    ['a', 'c', 'd'],\
    ['a', 'c', 'e'],\
    ['a', 'd', 'e'],\
    ['b', 'c', 'e']
]

## Hoàn chỉnh thuật toán Apriori

- Trong phần này, chúng ta sẽ hoàn chỉnh việc cài đặt thuật toán Apriori.
- Mục tiêu: tìm các tập phổ biến trong `list_transactions`

In [44]:
def Frequent_Itemset_Mining_Apriori(list_transactions, list_items, min_sup):

    list_frequent_itemset = []
    
    # YOUR CODE HERE
    c_list = list_items
    l_list = []

    while len(c_list) != 0:
        l_list = []
        for item in c_list:
            if type(item) == str:
                if Check_Frequent_Itemset([item], list_transactions, min_sup) and [item] not in l_list:
                    l_list.append([item])
            else:
                if Check_Frequent_Itemset(item, list_transactions, min_sup) and item not in l_list:
                    l_list.append(item)

        l_list = sorted(l_list)
        if len(l_list) == 0:
            return list_frequent_itemset

        list_frequent_itemset = list_frequent_itemset + l_list

        c_list = []
        for i in range(len(l_list)):
            for j in range(i+1, len(l_list)):
                join = Cal_Self_Join(l_list[i], l_list[j])
                if join != []:
                    c_list.append(join)
                    
        c_list = Prune_By_Apriori(c_list, l_list)
        
    return list_frequent_itemset

In [46]:
# TEST

list_transactions_test = [
    ['a', 'b', 'c', 'd', 'e'],\
    ['c', 'd', 'e', 'f', 'g'],\
    ['a', 'b', 'c', 'd'],\
    ['b', 'c', 'd', 'e'],\
    ['a', 'd', 'e', 'f'],\
    ['a', 'b', 'c', 'e'],\
    ['b', 'c', 'e', 'f'],\
    ['a', 'b', 'g'],\
    ['a', 'b', 'c', 'e', 'f'],\
    ['a', 'c', 'd', 'e'],\
]

list_items_test = ['a', 'b', 'c', 'd', 'e', 'f', 'g']

min_support_test = 0.4

list_fim = Frequent_Itemset_Mining_Apriori(list_transactions_test, list_items_test, min_support_test)

print(list_fim)
assert list_fim == [
                    ['a'], ['b'], ['c'], ['d'], ['e'], ['f'], \
                    ['a', 'b'], ['a', 'c'], ['a', 'd'], ['a', 'e'], ['b', 'c'], ['b', 'e'], \
                    ['c', 'd'], ['c', 'e'], ['d', 'e'], ['e', 'f'], \
                    ['a', 'b', 'c'], ['a', 'c', 'e'], ['b', 'c', 'e'], ['c', 'd', 'e']
                   ]

[['a'], ['b'], ['c'], ['d'], ['e'], ['f'], ['a', 'b'], ['a', 'c'], ['a', 'd'], ['a', 'e'], ['b', 'c'], ['b', 'e'], ['c', 'd'], ['c', 'e'], ['d', 'e'], ['e', 'f'], ['a', 'b', 'c'], ['a', 'c', 'e'], ['b', 'c', 'e'], ['c', 'd', 'e']]


## Kiểm tra thuật toán Apirori trên tập dữ liệu siêu thị

- Cell bên dưới, chỉ có mục đích kiểm tra giá trị `list_transactions` và `list_items` vẫn chính xác và sẳn sàng cho quá trình khai thác.

In [47]:
assert len(list_transactions) == 9835

assert list_items == sorted(list((set(list_items))))
assert len(list_items) == 169

In [48]:
# Test on our dataset
min_support = 0.1

list_fim = Frequent_Itemset_Mining_Apriori(list_transactions, list_items, min_support)

print(list_fim)

[['bottled water'], ['other vegetables'], ['rolls/buns'], ['root vegetables'], ['soda'], ['tropical fruit'], ['whole milk'], ['yogurt']]


### Nhiệm vụ của sinh viên:

- Đưa ra hai nhận xét về tập phổ biến.
- Sinh viên có thể thay đổi giá trị `min_sup` để quan sát thêm các kết quả.

In [56]:
min_support = 0.05

list_fim = Frequent_Itemset_Mining_Apriori(list_transactions, list_items, min_support)

set_1item = []
set_2item = []
for itemset in list_fim:
    if len(itemset) == 1:
        set_1item.append(itemset)
    else:
        set_2item.append(itemset)

print('1 item: ', len(set_1item), ' -> ', set_1item, '\n')
print('2 item: ', len(set_2item), ' -> ', set_2item)

1 item:  28  ->  [['beef'], ['bottled beer'], ['bottled water'], ['brown bread'], ['butter'], ['canned beer'], ['citrus fruit'], ['coffee'], ['curd'], ['domestic eggs'], ['frankfurter'], ['fruit/vegetable juice'], ['margarine'], ['napkins'], ['newspapers'], ['other vegetables'], ['pastry'], ['pip fruit'], ['pork'], ['rolls/buns'], ['root vegetables'], ['sausage'], ['shopping bags'], ['soda'], ['tropical fruit'], ['whipped/sour cream'], ['whole milk'], ['yogurt']] 

2 item:  3  ->  [['other vegetables', 'whole milk'], ['rolls/buns', 'whole milk'], ['whole milk', 'yogurt']]


- Với min_sup = 0.1: 
    - Tập phổ biến chỉ có 8 itemset. 
    - Nhưng lại không có itemset nào có 2 sản phẩm trở lên cả. 
    - Số lượng itemset gồm có 2 sản phẩm chưa đạt 10%
    - Không tìm được 2 sản phẩm để trưng bày gần nhau cho khách hàng mua
- Với min_sup = 0.05:
    - Lần này có đến 28 itemset có 1 sản phẩm
    - Có 3 itemset có 2 sản phẩm đi với nhau.
    - Cửa hàng có thể để 2 sản phẩm cùng nằm trong 1 itemset với nhau để bán hàng.
    - Tuy nhiên xác suất để chúng được mua cùng với nhau cũng không cao ~ 5% nên phải cân nhắc kĩ hơn để đưa ra sự lựa chọn
